# TASK_2

# Movie Rating Prediction

In [51]:
#import necessary libraries

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [6]:
# Load the movie dataset

df= pd.read_csv(r'IMDb Movies India.csv',encoding='latin')
df

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


# Data Preprocessing

In [7]:
# Handle missing values
df.dropna(inplace=True)

In [8]:
df

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,...Aur Pyaar Ho Gaya,(1997),147 min,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,...Yahaan,(2005),142 min,"Drama, Romance, War",7.4,"1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
8,?: A Question Mark,(2012),82 min,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia
...,...,...,...,...,...,...,...,...,...,...
15493,Zubaan,(2015),115 min,Drama,6.1,408,Mozez Singh,Vicky Kaushal,Sarah Jane Dias,Raaghavv Chanana
15494,Zubeidaa,(2001),153 min,"Biography, Drama, History",6.2,"1,496",Shyam Benegal,Karisma Kapoor,Rekha,Manoj Bajpayee
15503,Zulm Ki Zanjeer,(1989),125 min,"Action, Crime, Drama",5.8,44,S.P. Muthuraman,Chiranjeevi,Jayamalini,Rajinikanth
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani


In [9]:
df.head(10)

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
5,...Aur Pyaar Ho Gaya,(1997),147 min,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
6,...Yahaan,(2005),142 min,"Drama, Romance, War",7.4,"1,086",Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
8,?: A Question Mark,(2012),82 min,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia
9,@Andheri,(2014),116 min,"Action, Crime, Thriller",4.0,11,Biju Bhaskar Nair,Augustine,Fathima Babu,Byon
10,1:1.6 An Ode to Lost Love,(2004),96 min,Drama,6.2,17,Madhu Ambat,Rati Agnihotri,Gulshan Grover,Atul Kulkarni
11,1:13:7 Ek Tera Saath,(2016),120 min,Horror,5.9,59,Arshad Siddiqui,Pankaj Berry,Anubhav Dhir,Hritu Dudani
12,100 Days,(1991),161 min,"Horror, Romance, Thriller",6.5,983,Partho Ghosh,Jackie Shroff,Madhuri Dixit,Javed Jaffrey
13,100% Love,(2012),166 min,"Comedy, Drama, Romance",5.7,512,Rabi Kinagi,Jeet,Koyel Mallick,Sujoy Ghosh


In [10]:
df.columns

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')

In [11]:
df.shape

(5659, 10)

In [13]:
df.value_counts()

Name                                Year    Duration  Genre                      Rating  Votes  Director         Actor 1            Actor 2              Actor 3             
#Gadhvi (He thought he was Gandhi)  (2019)  109 min   Drama                      7.0     8      Gaurav Bakshi    Rasika Dugal       Vivek Ghamande       Arvind Jangid           1
Naya Andaz                          (1956)  142 min   Romance                    5.8     12     K. Amarnath      Kishore Kumar      Meena Kumari         Kumkum                  1
Number Game                         (2018)  117 min   Horror, Mystery, Thriller  6.3     10     Sumit Sinha      Aishwaryaa Rajesh  Nakul Choudharry     Rimi                    1
Noukadubi                           (2011)  135 min   Drama, Romance             7.6     845    Rituparno Ghosh  Jisshu Sengupta    Riya Sen             Raima Sen               1
Notebook                            (2019)  115 min   Drama, Romance             6.9     2,253  Nitin Kakkar  

In [14]:
df.duplicated().sum()

0

In [15]:
len(df)

5659

# Data Splitting


In [22]:
# Split the dataset into features (X) and target variable (y)
X = df[['Genre', 'Director', 'Actor 1','Actor 2','Actor 3']]
y = df['Rating']


In [23]:
print(X)
print(y)

                           Genre         Director          Actor 1  \
1                          Drama    Gaurav Bakshi     Rasika Dugal   
3                Comedy, Romance       Ovais Khan          Prateik   
5         Comedy, Drama, Musical     Rahul Rawail       Bobby Deol   
6            Drama, Romance, War   Shoojit Sircar  Jimmy Sheirgill   
8      Horror, Mystery, Thriller    Allyson Patel        Yash Dave   
...                          ...              ...              ...   
15493                      Drama      Mozez Singh    Vicky Kaushal   
15494  Biography, Drama, History    Shyam Benegal   Karisma Kapoor   
15503       Action, Crime, Drama  S.P. Muthuraman      Chiranjeevi   
15505              Action, Drama       Kuku Kohli     Akshay Kumar   
15508              Action, Drama     K.C. Bokadia       Dharmendra   

                      Actor 2           Actor 3  
1              Vivek Ghamande     Arvind Jangid  
3                  Ishita Raj   Siddhant Kapoor  
5      Ai

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Selection and Fine-Tuning

# Model Selection and Fine-Tuning:
- The code selects a regression model (Linear Regression in this case) and creates a pipeline using Pipeline from scikit-learn.
- Hyperparameters for the model are defined for fine-tuning. 
- In this example, the normalize hyperparameter of the linear regression model is fine-tuned using grid search cross-validation.
- GridSearchCV is used to perform grid search cross-validation over the defined hyperparameters.

In [42]:
# Define the preprocessor with OneHotEncoder (handle_unknown='ignore')
preprocessor = ColumnTransformer(
    transformers=[
        ('genre', OneHotEncoder(handle_unknown='ignore'), ['Genre']),
        ('director', OneHotEncoder(handle_unknown='ignore'), ['Director']),
        ('actor1', OneHotEncoder(handle_unknown='ignore'), ['Actor 1']) ,     
        ('actor2', OneHotEncoder(handle_unknown='ignore'), ['Actor 2']),
        ('actor3', OneHotEncoder(handle_unknown='ignore'), ['Actor 3']),

    ],
    remainder='passthrough'
)


In [43]:
# Update the pipeline with the preprocessor
model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [44]:
# Define hyperparameters for fine-tuning
param_grid = {
    'regressor__normalize': [True, False]
}

In [45]:
# Perform grid search cross-validation for fine-tuning
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('genre',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Genre']),
                                                                        ('director',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Director']),
                                                                        ('actor1',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Actor

In [46]:
# Get the best model from grid search
best_model = grid_search.best_estimator_
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre']),
                                                 ('director',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Director']),
                                                 ('actor1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Actor 1']),
                                                 ('actor2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Actor 2']),
                                                 ('

In [47]:
# Model Training (using best model)
best_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('genre',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Genre']),
                                                 ('director',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Director']),
                                                 ('actor1',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Actor 1']),
                                                 ('actor2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Actor 2']),
                                                 ('

# Model evaluation

In [48]:
# Evaluate the best model on the testing set
y_pred = best_model.predict(X_test)


In [49]:
y_pred

array([3.14005179, 5.92925178, 1.83957213, ..., 0.79376019, 8.97527566,
       5.82150636])

In [54]:
y_test

10971    6.0
14052    2.4
10002    3.8
3970     3.8
8840     7.2
        ... 
13713    6.7
2820     6.8
1810     4.8
13679    6.7
1649     6.6
Name: Rating, Length: 1132, dtype: float64

# MSE

In [55]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)


Mean Squared Error (MSE): 10.189402747147435
